In [1]:
%env XDG_CACHE=/workspace/.cache
%env HF_HOME=/workspace/.cache/huggingface

env: XDG_CACHE=/workspace/.cache
env: HF_HOME=/workspace/.cache/huggingface


In [2]:
from datasets import load_dataset
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import pandas as pd
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
import pyonmttok
import ctranslate2
from metrics import *

/workspace/poligloteval/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
model_id = "projecte-aina/aguila-7b"
#model_id = "tiiuae/falcon-7b"
model_name = model_id.split('/')[1]

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             torch_dtype=torch.bfloat16,
                                             trust_remote_code=True,
                                             device_map="auto")

## Lets Do the translation layer
from huggingface_hub import snapshot_download
print("Loading translator Models...")

ca_en_model_folder = snapshot_download(repo_id="projecte-aina/mt-aina-ca-en", revision="main")
tokenizer_ca_en = pyonmttok.Tokenizer(
    mode="none", sp_model_path=ca_en_model_folder + "/spm.model"
)
ca_en_model = ctranslate2.Translator(ca_en_model_folder, device="cuda")

Loading checkpoint shards: 100%|██████████| 15/15 [00:10<00:00,  1.39it/s]


Loading translator Models...


Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 23147.37it/s]


In [4]:
def min_max_scaling(tensor):
    min_val = torch.min(tensor)
    max_val = torch.max(tensor)
    scaled_tensor = (tensor - min_val) / (max_val - min_val)
    return scaled_tensor


def compute_probability(input_text, answer):
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    answer_tokens = tokenizer(answer)['input_ids']
    answer_probability = 1
    with torch.no_grad():
        for token in answer_tokens:
            outputs = model(**inputs, output_hidden_states=True, return_dict=True)

            # Logits are in the outputs, you can access the last token's logits like this:
            logits = outputs.logits[:, -1, :]
            log_probs = torch.log_softmax(logits, dim=-1).cpu()
            answer_probability += log_probs[0][token]

            # Prepare input_ids for the next token prediction
            new_token = torch.tensor([[token]]).to(model.device)
            inputs = {'input_ids': torch.cat([inputs['input_ids'], new_token], dim=1),
                    'attention_mask': torch.cat([inputs['attention_mask'], torch.tensor([[1]]).to(model.device)], dim=1)}
    return torch.exp(answer_probability).item()


def run_inference(txt, num_tokens=20, stop_text='\n'):
    # Tokenize the input text
    inputs = tokenizer(txt, return_tensors="pt").to(model.device)
    # Calculate the total length of the output (input length + number of tokens to generate)
    max_length = len(inputs['input_ids'][0]) + num_tokens

    with torch.no_grad():
        # Generate tokens
        tokens = model.generate(**inputs, do_sample=True, top_k=1, eos_token_id=tokenizer.eos_token_id, max_length=max_length)

        # Decode the generated tokens into text
        generated_text = tokenizer.decode(tokens[0], skip_special_tokens=True)

        # Slice the generated text to exclude the input prompt
        generated_only = generated_text[len(tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)):]

        # If a stop text is found, truncate the output at its first occurrence
        if stop_text in generated_only:
            generated_only = generated_only.split(stop_text)[0]

        return generated_only.strip()


def translate(sample):
    def translate_to_english(txt):
        lines = txt.split("\n")
        toks, _ = tokenizer_ca_en.tokenize_batch(lines)
        translated = ca_en_model.translate_batch(toks)
        ts = []
        for t in translated:
            ts.append(tokenizer_ca_en.detokenize(t.hypotheses[0]))

        return "\n".join(ts)
    en_prompt = translate_to_english(sample['prompt'])
    en_answer = translate_to_english(sample['answer'])
    return {"prompt": en_prompt, "answer": en_answer}


def compute_metrics(sample):
    prob = compute_probability(sample['prompt'], sample['answer'])
    prediction = run_inference(sample['prompt'])
    f1 = f1_score(prediction, sample['answer'])
    return {"prediction": prediction, "prob": prob, "f1": f1}


In [5]:
teca = load_dataset("data", data_files="teca.csv", split="train[:10]")
teca_en = teca.map(translate)

Parameter 'function'=<function translate at 0x7fc142290ee0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map:  10%|█         | 1/10 [00:00<00:01,  5.85 examples/s]

Map: 100%|██████████| 10/10 [00:01<00:00,  7.45 examples/s]


In [6]:
def eval(entry):
    prediction = run_inference(entry['prompt'], num_tokens=20, stop_text=' ')
    return { 'prediction': prediction, 'results': prediction == str(entry['answer']) }

results_ca = teca.map(eval)
results_ca.to_pandas()

Map:   0%|          | 0/10 [00:00<?, ? examples/s]/workspace/poligloteval/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Map: 100%|██████████| 10/10 [00:15<00:00,  1.57s/ examples]


,prompt,answer,numeric_label,prediction,results
0,"premissa: Nosaltres, per exemple, no estem d'a...",implicació,0,----\npremissa:,False
1,premissa: Els CDR convoquen una protesta davan...,contradicció,2,----\npremissa:,False
2,premissa: Vaig incorporar me a la feina quan e...,contradicció,2,----\n\npremissa:,False
3,premissa: La tradició i la modernitat conviuen...,neutre,1,----\n\npremissa:,False
4,premissa: Senda ofereix pràctiques d'un períod...,implicació,0,----\n\npremissa:,False
5,premissa: El doctor Josep Morata Socias i la p...,contradicció,2,----------\npremissa:,False
6,premissa: Això és el que passa quan la teva ge...,neutre,1,,False
7,premissa: M'ha sabut molt greu però no hi he p...,neutre,1,?natural\n----\npremissa:,False
8,premissa: El Concurs ha aconseguit gran presti...,neutre,1,----\n\npremissa:,False
9,premissa: Però l'altre dia llegint a la biblio...,contradicció,2,----------\n\npremissa:,False


In [7]:
results_en = teca_en.map(eval)
results_en.to_pandas()

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map: 100%|██████████| 10/10 [00:16<00:00,  1.60s/ examples]


,prompt,answer,numeric_label,prediction,results
0,"Premise: We, for example, do not agree to be t...",involvement in,0,-,False
1,premise: The CDRs call for a protest in front ...,contradiction in the law,2,,False
2,Premise: I joined the work when it was decided...,contradiction in the law,2,,False
3,premise: Tradition and modernity coexist in th...,neutral neutral,1,-,False
4,Premise: Senda offers internships for a minimu...,involvement in,0,-,False
5,premise: Dr. Josep Morata Socias and Professor...,contradiction in the law,2,,False
6,premise: This is what happens when your sister...,neutral neutral,1,,False
7,Premise: I was very sorry but I could not do a...,neutral neutral,1,,False
8,Premise: The Contest has achieved great presti...,neutral neutral,1,,False
9,Premise: But the other day reading in the libr...,contradiction in the law,2,-,False


In [8]:
results_ca.to_csv(f"results/{model_name}-teca-ca.csv", index=False)
results_en.to_csv(f"results/{model_name}-teca-en.csv", index=False)

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 377.29ba/s]


12089

In [9]:
print("Percentage correct ca:", sum(results_ca['results']) / len(results_ca))
print("Percentage correct en:", sum(results_en['results']) / len(results_en))

Percentage correct ca: 0.0
Percentage correct en: 0.0
